# $P(U|x)$ Calculations

In [1]:
# imports
import os
from pkg_resources import resource_filename
from importlib import reload

import numpy as np

import pandas

from astropy.coordinates import SkyCoord
#from astropy.io import fits

from astropath import bayesian

# Dummy FRB (based on 121102)

In [2]:
frb_coord = SkyCoord('05h31m58.7013s +33d08m52.5536s', frame='icrs')
frb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (82.99458875, 33.14793156)>

In [3]:
eellipse = dict(a=0.1, b=0.1, theta=0.)

# Galaxies

In [4]:
cand_file = os.path.join(resource_filename('astropath', 'data'), 'frb_example', 'frb121102_candidates.csv')

In [5]:
candidates = pandas.read_csv(cand_file, index_col=0)
candidates.head()

,ra,dec,half_light,GMOS_N_i
3,82.991765,33.146968,0.254012,23.529823
4,82.992292,33.146890,0.277451,21.584045
6,82.993687,33.145270,0.307746,20.190046
7,82.993496,33.147349,0.229700,24.180188
8,82.993928,33.149159,0.132098,25.062593


In [6]:
c_coords = SkyCoord(ra=candidates.ra, dec=candidates.dec, unit='deg')
c_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(82.99176537, 33.14696783), (82.99229246, 33.14688994),
     (82.99368705, 33.14526968), (82.99349587, 33.14734905),
     (82.9939283 , 33.14915907), (82.99423121, 33.14718576),
     (82.9945254 , 33.14791209), (82.99484463, 33.15027849),
     (82.996021  , 33.14846041), (82.99680541, 33.14898092)]>

In [7]:
candidates['sep'] = frb_coord.separation(c_coords).to('arcsec').value

# Priors

## Guiding dict

In [8]:
offset_prior = dict(method='uniform', 
                    max=6.,
                   ang_size=candidates.half_light.values)
priors = dict(offset=offset_prior, 
              O='identical', 
              U=0.10, 
              name='Conservative')
box_radius = 30. # arcsec

## Raw

In [9]:
# Raw priors
raw_prior_Oi = bayesian.raw_prior_Oi(priors['O'], 
                                     candidates.GMOS_N_i.values, 
                                     half_light=candidates.half_light.values)

In [10]:
candidates['P_O_raw'] = raw_prior_Oi
candidates

,ra,dec,half_light,GMOS_N_i,sep,P_O_raw
3,82.991765,33.146968,0.254012,23.529823,9.190131,1.0
4,82.992292,33.146890,0.277451,21.584045,7.871884,1.0
6,82.993687,33.145270,0.307746,20.190046,9.960715,1.0
7,82.993496,33.147349,0.229700,24.180188,3.904953,1.0
8,82.993928,33.149159,0.132098,25.062593,4.846743,1.0
9,82.994231,33.147186,0.279821,21.139298,2.893068,1.0
10,82.994525,33.147912,0.282400,23.516754,0.203389,1.0
11,82.994845,33.150278,0.147827,24.954572,8.484102,1.0
12,82.996021,33.148460,0.254041,23.278666,4.718179,1.0
14,82.996805,33.148981,0.240684,22.907361,7.675323,1.0


## Normalize -- With $P(U) = 0.10$

In [11]:
candidates['P_O'] = bayesian.renorm_priors(candidates.P_O_raw.values, priors['U'])
candidates

,ra,dec,half_light,GMOS_N_i,sep,P_O_raw,P_O
3,82.991765,33.146968,0.254012,23.529823,9.190131,1.0,0.09
4,82.992292,33.146890,0.277451,21.584045,7.871884,1.0,0.09
6,82.993687,33.145270,0.307746,20.190046,9.960715,1.0,0.09
7,82.993496,33.147349,0.229700,24.180188,3.904953,1.0,0.09
8,82.993928,33.149159,0.132098,25.062593,4.846743,1.0,0.09
9,82.994231,33.147186,0.279821,21.139298,2.893068,1.0,0.09
10,82.994525,33.147912,0.282400,23.516754,0.203389,1.0,0.09
11,82.994845,33.150278,0.147827,24.954572,8.484102,1.0,0.09
12,82.996021,33.148460,0.254041,23.278666,4.718179,1.0,0.09
14,82.996805,33.148981,0.240684,22.907361,7.675323,1.0,0.09


In [12]:
np.sum(candidates.P_O)

0.8999999999999999

# $p(x)$

## $p(x|O)$

In [13]:
p_xOi = bayesian.px_Oi(box_radius,  # box radius for grid, in arcsec
                       frb_coord,
                       eellipse,
                       c_coords,
                       priors['offset'])
p_xOi

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 4.40181892e-140,
       0.00000000e+000, 5.72035937e-033, 5.88213788e+001, 0.00000000e+000,
       2.35854039e-223, 0.00000000e+000])

In [14]:
candidates['p_xO'] = p_xOi

## $p(x|U)$

In [15]:
reload(bayesian)
p_xU = bayesian.px_U(box_radius)
p_xU

940454.7440720972

In [21]:
p_x = np.sum(candidates.P_O * candidates.p_xO) + p_xU*priors['U']
p_x

94050.76833129757

# Posteriors

## Observed

In [22]:
P_Oix = candidates.P_O * p_xOi / p_x
candidates['P_Ox'] = P_Oix

In [23]:
candidates

,ra,dec,half_light,GMOS_N_i,sep,P_O_raw,P_O,p_xO,P_Ox
3,82.991765,33.146968,0.254012,23.529823,9.190131,1.0,0.09,0.000000e+00,0.000000e+00
4,82.992292,33.146890,0.277451,21.584045,7.871884,1.0,0.09,0.000000e+00,0.000000e+00
6,82.993687,33.145270,0.307746,20.190046,9.960715,1.0,0.09,0.000000e+00,0.000000e+00
7,82.993496,33.147349,0.229700,24.180188,3.904953,1.0,0.09,4.401819e-140,4.212233e-146
8,82.993928,33.149159,0.132098,25.062593,4.846743,1.0,0.09,0.000000e+00,0.000000e+00
9,82.994231,33.147186,0.279821,21.139298,2.893068,1.0,0.09,5.720359e-33,5.473983e-39
10,82.994525,33.147912,0.282400,23.516754,0.203389,1.0,0.09,5.882138e+01,5.628794e-05
11,82.994845,33.150278,0.147827,24.954572,8.484102,1.0,0.09,0.000000e+00,0.000000e+00
12,82.996021,33.148460,0.254041,23.278666,4.718179,1.0,0.09,2.358540e-223,2.256958e-229
14,82.996805,33.148981,0.240684,22.907361,7.675323,1.0,0.09,0.000000e+00,0.000000e+00


## Unseen

In [24]:
P_Ux = priors['U'] * p_xU / p_x
P_Ux

0.9999437120591168

----

In [20]:
np.pi * 4 * (180./np.pi)**2

41252.96124941928